### Rate Limits

- 20 requests every 1 seconds 
- 100 requests every 2 minutes

### First get 10000 match histories.

- First 10 columns are the champions for blue team, red team and the last columns stands for "Blue team won"

In [1]:
import cassiopeia as cass
import getpass
import random

key = getpass.getpass("Enter Riot API key:")

Enter Riot API key:········


In [2]:
cass.set_riot_api_key(key)  # This overrides the value set in your configuration/settings.
cass.set_default_region("KR")

In [3]:
match = cass.get_match(3492500002)

In [4]:
(match.blue_team.participants[0]).summoner.ranks[match.queue]

Making call: https://ddragon.leagueoflegends.com/realms/kr.json
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/3492500002
Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by-summoner/YeE2YgdwrGeM9m1b4dH-xPrJiA8ovJEnhJVbRfleQdNLAgU


In [5]:
import pickle as pkl
import os
import pandas as pd
from cassiopeia.datastores.riotapi.common import APIRequestError, APIError
from datapipelines.pipelines import NotFoundError


In [6]:
def get_lane(participant):
    try:
        lane = participant.lane.name
        if lane =='bot_lane':
            lane = participant.role.name
        return lane
    except AttributeError:
        return 'No info'

In [7]:
def get_data(i):
    dd = []
    match = cass.get_match(i)
    try:
        version = match.version
        queue = match.queue.value
    except APIRequestError:
        print('Your API key has expired.')
        key = getpass.getpass("Enter new Riot API key:")
        cass.set_riot_api_key(key)
        try:
            version = match.version
            queue = match.queue.value
        except:
            return []
    except:
        return []
        
    if queue in ['RANKED_SOLO_5x5'] and version == '9.1.257.7563':        
        for participant in match.blue_team.participants :
            try:
                lane = get_lane(participant)
                champion = participant.champion.name
                rank = participant.summoner.ranks[match.queue]
                tier = ' '.join([rank.tier.name, rank.division.name])
            except APIRequestError:
                print('Your API key has expired.')
                key = getpass.getpass("Enter new Riot API key:")
                cass.set_riot_api_key(key)
                lane = get_lane(participant)
                champion = participant.champion.name
                rank = participant.summoner.ranks[match.queue]
                tier = ' '.join([rank.tier.name, rank.division.name])
            
            dd.append((lane, participant.champion.name, tier))
            
        for ban in match.blue_team.bans:
            try:
                dd.append(ban.name)
            except AttributeError:
                dd.append('밴없음')
                
        for participant in match.red_team.participants :
            try:
                lane = get_lane(participant)
                champion = participant.champion.name
                rank = participant.summoner.ranks[match.queue]
                tier = ' '.join([rank.tier.name, rank.division.name])
            except APIRequestError:
                print('Your API key has expired.')
                key = getpass.getpass("Enter new Riot API key:")
                cass.set_riot_api_key(key)
                lane = get_lane(participant)
                champion = participant.champion.name
                rank = participant.summoner.ranks[match.queue]
                tier = ' '.join([rank.tier.name, rank.division.name])
            
            dd.append((lane, participant.champion.name, tier))
            
        for ban in match.red_team.bans:
            try:
                dd.append(ban.name)
            except AttributeError:
                dd.append('밴없음')

        dd.append(match.blue_team.win)
        print('Match id {} is valid.'.format(i))
    return dd

# Set start_id and end_id

||start_id|end_id|
|----------|------------|------------|
| Jaewoo   | 3490000000 | 3492500000 |
| Jinyoung | 3492500001 | 3495000000 |
| Hwi      | 3495000001 | 3475000000 |
| Q        | 3475000001 | 3500000000 |

In [8]:
start_id = 3492500001
end_id = 3495000000
ids = range(start_id, end_id)

dir_name = '{}-{}'.format(start_id, end_id)
os.mkdir(os.path.join('../data/',dir_name))

data = []
user = os.path.realpath('.').split('/')[2]
k=0

In [ ]:
for i in ids:
    try:
        dd = get_data(i)
        if len(dd)!=0:
            data.append(dd)
        if len(data)==2000:
            print("Making {}.pkl.".format(k))
            with open(os.path.join('../data/', dir_name, '{}.pkl'.format(k)), 'wb') as f:
                pkl.dump(data, f)
            k+=1
            data = []
    except:
        print('Error occured for {}'.format(i))
        continue

if len(data)!=0:
    with open(os.path.join('../data/', dir_name, '{}.pkl'.format(k)), 'wb') as f:
        pkl.dump(data, f)
        

Making call: https://kr.api.riotgames.com/lol/match/v4/matches/3492500001
Making call: https://ddragon.leagueoflegends.com/cdn/9.1.1/data/ko_KR/championFull.json
Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by-summoner/4SkHNVTh2cStL2bo_0sZe9wdfn55Kj-tYL4glvbZ3QEy85I
Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by-summoner/tz140m4nvgh7Z6po6sb-KI6zTW1RCsdD12b3QdAeLqNn9Q
Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by-summoner/teM6qGcPOV1RmcgW6tmtQ5WtjdlaKkMNPgPx9hFM3r0DpQ0
Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by-summoner/xRibK94j_1YUTIa96ioPpYtdDRBv92jr7EDqkNY37jsVKNI
Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by-summoner/1NuPRHHfoaWDgrmI0rdUtYOjemjKmjq4KrW2aKzhyH3A2w
Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by-summoner/Nfvoyzj6OHOXy08KzOfCSujSQpCnjLx3DK_BHZ_7ye7Eqg
Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by-summoner/VINKMr1

Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by-summoner/DG4NtV4auv2n9WFsHmTJhIuMLKlE-D8ISVImKdWJNtmSOw
Match id 3492500028 is valid.
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/3492500029
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/3492500030
Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by-summoner/ChYPCimp2bp1Mg3Yvi6DitSE_NXxV0tPCKm9IQCFx_IXKzE
Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by-summoner/OcOXgWsMRbCswlTRQPARZBO_rhm_lBKYySET43z4LoqEPs4
Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by-summoner/RguBluhQ9QvLyaNYfXnusSDXWX8xWzU6zgngtf5JDqVyCA
Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by-summoner/K-mlsm-4l40-yRw8TXkjp7PsVsaMmJsCeLoS_ZI5RazRYW4
Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by-summoner/nUa0J4BJxxHvWn3dzlWWKXFF0G6GPkybu-o_2LTyNaLpJbQ
Making call: https://kr.api.riotgames.com/lol/league/v4/positions/by